# Goals

* Check whether Chris' datasets are in the scBaseCount SQL database (STAR results)
* Also created `prodC` on GCP for Chris' datasets

In [1]:
import os
from pathlib import Path
import pandas as pd
import plotnine as pn
from pypika import Query, Table, functions as fn

from SRAgent.db.connect import db_connect

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 4)
pn.theme_set(pn.theme_bw())

In [3]:
# get base of github repo
base_dir = !git rev-parse --show-toplevel
base_dir = Path(base_dir[0])
base_dir

PosixPath('/home/nickyoungblut/dev/python/scBaseCount_analysis')

In [4]:
# set tenant
os.environ["DYNACONF"] = "prod"

In [5]:
def get_screcounter_star_results(
    conn, 
    feature="GeneFull_Ex50pAS"
):
    """
    Get the screcounter_star_results table
    
    Args:
        conn: A database connection object (e.g., sqlite3, psycopg2, etc.)
        feature: String of the feature to filter on
    Returns:
        A pandas DataFrame of records
    """
    
    # Define tables
    meta_table = Table('srx_metadata')
    star_table = Table('screcounter_star_results')
    
    # Build the star query with feature filter (case insensitive)
    star_query = (
        Query.from_(star_table)
        .select('*')
        .where(
            fn.Lower(star_table.feature) == feature.lower()
        )
    )

    # Build the main query with join
    # Assuming you want all columns from both tables except feature from star and _at columns from meta
    main_query = (
        Query.from_(meta_table)
        .left_join(star_table)
        .on(meta_table.srx_accession == star_table.sample)
        .select(
            star_table.sample.as_('srx_accession'),
            star_table.feature,
            star_table.estimated_number_of_cells,
            meta_table.lib_prep,
            meta_table.tech_10x,
            meta_table.organism,
            meta_table.tissue,
            meta_table.tissue_ontology_term_id,
            meta_table.czi_collection_id,
            meta_table.czi_collection_name,
            meta_table.notes
        )
        .where(
            meta_table.czi_collection_id != None and meta_table.czi_collection_id != ""
        )
        .where(
            star_table.estimated_number_of_cells.isnull()
        )
    )
    
    # Execute query and return as DataFrame
    return pd.read_sql_query(str(main_query), conn)

with db_connect() as conn:
    df_cells = get_screcounter_star_results(conn, feature="GeneFull_Ex50pAS")
df_cells

,srx_accession,feature,estimated_number_of_cells,lib_prep,tech_10x,organism,tissue,tissue_ontology_term_id,czi_collection_id,czi_collection_name,notes
0,None,None,None,other,not_applicable,Homo sapiens,brain,None,999f2a15-3d7e-440b-96ae-2c806799c08c,"Harmonized single-cell landscape, intercellula...",Metadata obtained by SRAgent
1,None,None,None,other,not_applicable,Homo sapiens,acute slice culture of glioma resection,None,999f2a15-3d7e-440b-96ae-2c806799c08c,"Harmonized single-cell landscape, intercellula...",Metadata obtained by SRAgent
...,...,...,...,...,...,...,...,...,...,...,...
2380,None,None,None,other,not_applicable,Homo sapiens,tumor tissue,None,3f50314f-bdc9-40c6-8e4a-b0901ebfbe4c,Single-cell sequencing links multiregional imm...,Metadata obtained by SRAgent
2381,None,None,None,other,not_applicable,Homo sapiens,H1299 cells,None,d0e9c47b-4ce7-4f84-b182-eddcfa0b2658,Single-cell gene expression profiling of SARS-...,Metadata obtained by SRAgent


# Location of Chris' dataset

In [6]:
data_dir = Path('/processed_datasets/scRecount/cellxgene/counted_SRXs/')
data_dir

PosixPath('/processed_datasets/scRecount/cellxgene/counted_SRXs')

In [7]:
# find all matrix files
matrix_info = []
for f in data_dir.rglob("matrix.mtx.gz"):
    accession = f.parts[5]
    feature = f.parts[7]
    processing = f.parent.name
    matrix_info.append([accession, feature, processing, f])

matrix_info = pd.DataFrame(matrix_info, columns=["accession", "feature", "processing", "path"])
matrix_info

,accession,feature,processing,path
0,ERX8792024,GeneFull_ExonOverIntron,filtered,/processed_datasets/scRecount/cellxgene/counte...
1,ERX8792024,GeneFull_ExonOverIntron,raw,/processed_datasets/scRecount/cellxgene/counte...
...,...,...,...,...
22895,ERX11467033,GeneFull_Ex50pAS,raw,/processed_datasets/scRecount/cellxgene/counte...
22896,ERX11467033,GeneFull_Ex50pAS,filtered,/processed_datasets/scRecount/cellxgene/counte...


In [8]:
# high priority matrix files
matrix_info[(matrix_info["processing"] == "filtered") & (matrix_info["feature"] == "GeneFull_Ex50pAS")]

,accession,feature,processing,path
4,ERX8792024,GeneFull_Ex50pAS,filtered,/processed_datasets/scRecount/cellxgene/counte...
10,ERX12558782,GeneFull_Ex50pAS,filtered,/processed_datasets/scRecount/cellxgene/counte...
...,...,...,...,...
22868,ERX9138882,GeneFull_Ex50pAS,filtered,/processed_datasets/scRecount/cellxgene/counte...
22896,ERX11467033,GeneFull_Ex50pAS,filtered,/processed_datasets/scRecount/cellxgene/counte...


In [9]:
# find all summary files
sum_info = []
for f in data_dir.rglob("Summary.csv"):
    feature = f.parent.stem
    accession = f.parent.parent.parent.stem
    sum_info.append([accession, feature, f])

sum_info = pd.DataFrame(sum_info, columns=["accession", "feature", "path"])
sum_info

,accession,feature,path
0,ERX8792024,GeneFull_ExonOverIntron,/processed_datasets/scRecount/cellxgene/counte...
1,ERX8792024,GeneFull,/processed_datasets/scRecount/cellxgene/counte...
...,...,...,...
14675,ERX11467033,Gene,/processed_datasets/scRecount/cellxgene/counte...
14676,ERX11467033,GeneFull_Ex50pAS,/processed_datasets/scRecount/cellxgene/counte...


In [10]:
# filter sum_info to just those with filtered matrix
sum_info_f = sum_info[sum_info["accession"].isin(matrix_info["accession"])]
sum_info_f

,accession,feature,path
0,ERX8792024,GeneFull_ExonOverIntron,/processed_datasets/scRecount/cellxgene/counte...
1,ERX8792024,GeneFull,/processed_datasets/scRecount/cellxgene/counte...
...,...,...,...
14675,ERX11467033,Gene,/processed_datasets/scRecount/cellxgene/counte...
14676,ERX11467033,GeneFull_Ex50pAS,/processed_datasets/scRecount/cellxgene/counte...


In [11]:
# number of unique SRXs
sum_info_f["accession"].nunique()

2179

In [12]:
# find duplicates
x = sum_info_f[sum_info_f["feature"] == "GeneFull_Ex50pAS"]
df_dup = x[x["accession"].duplicated()].sort_values("accession")

pd.options.display.max_rows = 100
pd.options.display.max_colwidth = None
df_dup

,accession,feature,path


In [13]:
# reset display
pd.options.display.max_rows = 4

## Filter to those with metadata in the SQL database

In [14]:
target_srx = sum_info_f["accession"].unique().tolist()

meta_table = Table('srx_metadata')
query = (
    Query
    .from_(meta_table)
    .select(
        meta_table.srx_accession,
        meta_table.lib_prep,
        meta_table.tech_10x,
        meta_table.organism,
        meta_table.tissue,
        meta_table.tissue_ontology_term_id,
        meta_table.czi_collection_id,
        meta_table.czi_collection_name,
        meta_table.notes
    )
    .where(
        meta_table.srx_accession.isin(target_srx)
    )
)
    
with db_connect() as conn:
    df_in_db = pd.read_sql_query(str(query), conn)
df_in_db 

,srx_accession,lib_prep,tech_10x,organism,tissue,tissue_ontology_term_id,czi_collection_id,czi_collection_name,notes
0,SRX6640190,other,not_applicable,Homo sapiens,spleen,None,38833785-fac5-48fd-944a-0f62a4c23ed1,Construction of a human cell landscape at single-cell level,Metadata obtained by SRAgent
1,SRX6640168,other,not_applicable,Homo sapiens,lung,None,38833785-fac5-48fd-944a-0f62a4c23ed1,Construction of a human cell landscape at single-cell level,Metadata obtained by SRAgent
...,...,...,...,...,...,...,...,...,...
2170,SRX12677803,10x_Genomics,3_prime_gex,Mus musculus,thymus,UBERON:0002370,None,None,Metadata obtained by SRAgent
2171,SRX13549173,not_applicable,not_applicable,Mus musculus,liver,UBERON:0002107,None,None,Metadata obtained by SRAgent


In [15]:
# filter chris's data to those in the SQL database
x = df_in_db["srx_accession"].unique().tolist()
sum_info_ff = sum_info_f[sum_info_f["accession"].isin(x)]
sum_info_ff["accession"].nunique()

2172

### Accessions with no in metadata table of SQL database

In [16]:
# filter to those missing database
x = df_in_db["srx_accession"].unique().tolist()
sum_info_f_mis = sum_info_f[~sum_info_f["accession"].isin(x)]
sum_info_f_mis["accession"].nunique()

7

In [17]:
# write output
outfile = base_dir /  'data' / 'chris_data' / '2025-07-03_srx-add-meta.csv'
sum_info_f_mis.to_csv(outfile, index=False)
print(f"File written to {outfile}")

File written to /home/nickyoungblut/dev/python/scBaseCount_analysis/data/chris_data/2025-07-03_srx-add-meta.csv


> Note: the 7 are outliers that will not be added to the database

#

## Filter to just those lacking STAR results in the SQL database

In [18]:
target_srx = sum_info_ff["accession"].unique().tolist()

star_table = Table('screcounter_star_results')
query = (
    Query
    .from_(star_table)
    .select(
        star_table.sample
    )
    .distinct()
    .where(
        star_table.sample.isin(target_srx)
    )
)
    
with db_connect() as conn:
    df_has_star = pd.read_sql_query(str(query), conn)
df_has_star

,sample
0,ERX11467030
1,ERX11467031
...,...
739,SRX9777398
740,SRX9777399


In [19]:
# filter to just those lacking an SRA experiment accession; thus, should be added to the STAR results table
x = df_has_star["sample"].unique().tolist()
sum_info_ff = sum_info_ff[~sum_info_f["accession"].isin(x)]
sum_info_ff["accession"].nunique()

/tmp/ipykernel_848383/2693290625.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


1431

In [20]:
# check for duplidates
x = sum_info_ff.copy()
x = x[x["feature"] == "GeneFull_Ex50pAS"]
x[x["accession"].duplicated()]

,accession,feature,path


In [21]:
# check for all features per accession
pd.options.display.max_rows = 10
sum_info_ff.groupby("feature").count().reset_index()

,feature,accession,path
0,Gene,1431,1431
1,GeneFull,1431,1431
2,GeneFull_Ex50pAS,1431,1431
3,GeneFull_ExonOverIntron,1431,1431
4,Velocyto,1428,1428


In [22]:
# reset display
pd.options.display.max_rows = 4

In [23]:
# write output
outfile = base_dir /  'data' / 'chris_data' / '2025-07-03_srx-add-star.csv'
sum_info_ff.to_csv(outfile, index=False)
print(f"File written to {outfile}")

File written to /home/nickyoungblut/dev/python/scBaseCount_analysis/data/chris_data/2025-07-03_srx-add-star.csv


### Add to SQL database & `prodC` GCP bucket location

In [28]:
csv_file = base_dir / 'data' / 'chris_data' / '2025-07-03_srx-add-star.csv'
print(csv_file.exists())

exe = base_dir / 'scripts' / 'ChrisData2SQL-DB.py'
print(exe.exists())

True
True


#### Test run

In [37]:
slurm_job = f"""#!/bin/bash
#SBATCH --job-name=ChrisData2SQLDB  
#SBATCH --output=ChrisData2SQLDB.out
#SBATCH --error=ChrisData2SQLDB.err
#SBATCH --ntasks=1 
#SBATCH --cpus-per-task=1
#SBATCH --mem=8G
#SBATCH --time=72:00:00

{exe} \\
    --tenant test \\
    --max-datasets 2 \\
    --gcp-bucket gs://arc-ctc-screcounter/test/prodC/ \\
    --samples-table {csv_file}
"""

job_file = "ChrisData2SQL-DB.sh"
with open(job_file, "w") as f:
    f.write(slurm_job)

!chmod +x ChrisData2SQL-DB.sh
!cat ChrisData2SQL-DB.sh

#!/bin/bash
#SBATCH --job-name=ChrisData2SQLDB  
#SBATCH --output=ChrisData2SQLDB.out
#SBATCH --error=ChrisData2SQLDB.err
#SBATCH --ntasks=1 
#SBATCH --cpus-per-task=1
#SBATCH --mem=8G
#SBATCH --time=72:00:00

/home/nickyoungblut/dev/python/scBaseCount_analysis/scripts/ChrisData2SQL-DB.py \
    --tenant test \
    --max-datasets 2 \
    --gcp-bucket gs://arc-ctc-screcounter/test/prodC/ \
    --samples-table /home/nickyoungblut/dev/python/scBaseCount_analysis/data/chris_data/2025-07-03_srx-add-star.csv


In [ ]:
# submit job
!sbatch ChrisData2SQL-DB.sh

Submitted batch job 934890


In [46]:
!tail ChrisData2SQLDB.err

2025-07-03 09:23:12,981 - Successfully uploaded matrix files for ERX12558782/Gene/raw
2025-07-03 09:23:12,982 - Uploading matrix files: /processed_datasets/scRecount/cellxgene/counted_SRXs/ERX12558782/ERR13187479_resultSolo.out/Gene/filtered -> gs://arc-ctc-screcounter/test/prodC/SCRECOUNTER_2025-07-03_00-00-00/STAR/ERX12558782/Gene/filtered/
2025-07-03 09:23:14,845 - Successfully uploaded matrix files for ERX12558782/Gene/filtered
2025-07-03 09:23:14,846 - Uploading matrix files: /processed_datasets/scRecount/cellxgene/counted_SRXs/ERX12558782/ERR13187479_resultSolo.out/GeneFull_ExonOverIntron/raw -> gs://arc-ctc-screcounter/test/prodC/SCRECOUNTER_2025-07-03_00-00-00/STAR/ERX12558782/GeneFull_ExonOverIntron/raw/
2025-07-03 09:23:16,812 - Successfully uploaded matrix files for ERX12558782/GeneFull_ExonOverIntron/raw
2025-07-03 09:23:16,812 - Uploading matrix files: /processed_datasets/scRecount/cellxgene/counted_SRXs/ERX12558782/ERR13187479_resultSolo.out/GeneFull_ExonOverIntron/filter

#### Full run

In [60]:
slurm_job = f"""#!/bin/bash
#SBATCH --job-name=ChrisData2SQLDB  
#SBATCH --output=ChrisData2SQLDB.out
#SBATCH --error=ChrisData2SQLDB.err
#SBATCH --ntasks=1 
#SBATCH --cpus-per-task=1
#SBATCH --mem=12G
#SBATCH --time=96:00:00

{exe} \\
    --tenant prod \\
    --ignore-missing \\
    --gcp-bucket gs://arc-ctc-screcounter/prodC/ \\
    --samples-table {csv_file}
"""

job_file = "ChrisData2SQL-DB.sh"
with open(job_file, "w") as f:
    f.write(slurm_job)

!chmod +x ChrisData2SQL-DB.sh
!cat ChrisData2SQL-DB.sh

#!/bin/bash
#SBATCH --job-name=ChrisData2SQLDB  
#SBATCH --output=ChrisData2SQLDB.out
#SBATCH --error=ChrisData2SQLDB.err
#SBATCH --ntasks=1 
#SBATCH --cpus-per-task=1
#SBATCH --mem=12G
#SBATCH --time=96:00:00

/home/nickyoungblut/dev/python/scBaseCount_analysis/scripts/ChrisData2SQL-DB.py \
    --tenant prod \
    --ignore-missing \
    --gcp-bucket gs://arc-ctc-screcounter/prodC/ \
    --samples-table /home/nickyoungblut/dev/python/scBaseCount_analysis/data/chris_data/2025-07-03_srx-add-star.csv


In [61]:
# submit job
!sbatch ChrisData2SQL-DB.sh

Submitted batch job 935000


In [5]:
!tail -n 20 ChrisData2SQLDB.err

2025-07-03 13:09:09,871 - Successfully uploaded matrix files for ERX12098962/GeneFull_ExonOverIntron/raw
2025-07-03 13:09:09,871 - Uploading matrix files: /processed_datasets/scRecount/cellxgene/counted_SRXs/ERX12098962/ERR12724463_resultSolo.out/GeneFull_ExonOverIntron/filtered -> gs://arc-ctc-screcounter/prodC/SCRECOUNTER_2025-07-03_00-00-00/STAR/ERX12098962/GeneFull_ExonOverIntron/filtered/
2025-07-03 13:09:14,022 - Successfully uploaded matrix files for ERX12098962/GeneFull_ExonOverIntron/filtered
2025-07-03 13:09:14,023 - Uploading matrix files: /processed_datasets/scRecount/cellxgene/counted_SRXs/ERX12098962/ERR12724463_resultSolo.out/Gene/raw -> gs://arc-ctc-screcounter/prodC/SCRECOUNTER_2025-07-03_00-00-00/STAR/ERX12098962/Gene/raw/
2025-07-03 13:09:21,595 - Successfully uploaded matrix files for ERX12098962/Gene/raw
2025-07-03 13:09:21,597 - Uploading matrix files: /processed_datasets/scRecount/cellxgene/counted_SRXs/ERX12098962/ERR12724463_resultSolo.out/Gene/filtered -> gs:/

# Sandbox

In [10]:
os.environ["DYNACONF"] = "test"

In [13]:
target = 'ERX8792024'


srx_metadata = Table("srx_metadata")
stmt = Query \
    .from_(srx_metadata) \
    .select(
        srx_metadata.srx_accession,
        srx_metadata.database,
        srx_metadata.entrez_id,
    ) \
    .distinct() \
    .where(srx_metadata.srx_accession == target)
        
with db_connect() as conn:
    print(pd.read_sql_query(str(stmt), conn))

  srx_accession database  entrez_id
0    ERX8792024      sra   21270500


# session info

In [117]:
!conda list

# packages in environment at /home/nickyoungblut/miniforge3/envs/SRAgent_nb:
#
# Name                    Version                   Build  Channel
_libgcc_mutex             0.1                 conda_forge    conda-forge
_openmp_mutex             4.5                       2_gnu    conda-forge
_python_abi3_support      1.0                  hd8ed1ab_2    conda-forge
aiobotocore               2.23.0             pyhd8ed1ab_0    conda-forge
aiohappyeyeballs          2.6.1              pyhd8ed1ab_0    conda-forge
aiohttp                   3.11.14                  pypi_0    pypi
aioitertools              0.12.0             pyhd8ed1ab_1    conda-forge
aiosignal                 1.3.2              pyhd8ed1ab_0    conda-forge
anndata                   0.11.4             pyhd8ed1ab_0    conda-forge
annotated-types           0.7.0                    pypi_0    pypi
anyio                     4.9.0                    pypi_0    pypi
appdirs                   1.4.4                    pypi_0    pypi
array-